### Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the pandas documentation to find functions or methods you might not have used yet, or ask questions on Stack Overflow and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:

A quarter is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
A recession bottom is the quarter within a recession which had the lowest GDP.
A university town is a city which has a high percentage of university students compared to the total population of the city.
Hypothesis: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)

The following data files are available for this assignment:

From the Zillow research data site there is housing data for the United States. In particular the datafile for all homes at a city level, City_Zhvi_AllHomes.csv, has median home sale prices at a fine grained level.
From the Wikipedia page on college towns is a list of university towns in the United States which has been copy and pasted into the file university_towns.txt.
From Bureau of Economic Analysis, US Department of Commerce, the GDP over time of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file gdplev.xls. For this assignment, only look at GDP data from the first quarter of 2000 onward.
Each function in this assignment below is worth 10%, with the exception of run_ttest(), which is worth 50%.

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [25]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [215]:
ut = pd.read_table('data/university_towns.txt', header=None)
ut.shape


(567, 1)

In [3]:
def get_list_of_university_towns():
    map_region = {}
    states = []
    ut = pd.read_table('data/university_towns.txt', header=None)
    
    for index, row in ut.iterrows():
        rowStr = row[0]
        if 'edit' in rowStr:
            states.append(rowStr[:rowStr.find('[')])
        if 'edit' not in rowStr:
            key = rowStr
            map_region[rowStr] = states[-1]
    
    uni_town = pd.DataFrame(list(map_region.items()))
    uni_town.columns = ['RegionName', 'State']
    uni_town.RegionName = uni_town['RegionName'].apply(lambda x: x[:x.find(' (')])  
    
    uni_town.to_csv("data/university_towns.csv")
    return uni_town

get_list_of_university_towns().head()

,RegionName,State
0,Auburn,Alabama
1,Florence,Alabama
2,Jacksonville,Alabama
3,Livingston,Alabama
4,Montevallo,Alabama


In [73]:
gdplev = pd.read_excel('data/gdplev.xls', skiprows=5,)
gdplev.drop([0,1], inplace=True)
gdplev.drop(['Unnamed: 3', 'Unnamed: 7'], axis=1,inplace=True)
gdplev.columns = ['year', 'gdp in billions of current dollars', 'gdp in billions of chained 2009 dollars', 'year.1','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']

gdp_year = gdplev[['year', 'gdp in billions of current dollars', 'gdp in billions of chained 2009 dollars']]
gdp_year = gdp_year.loc[gdp_year.year > 0]
gdp_quarter = gdplev[['year.1','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']]



year.1                                       1947q3
gdp in billions of current dollars.1          250.1
gdp in billions of chained 2009 dollars.1    1930.3
Name: 4, dtype: object

In [4]:
def get_recession_start():
    gdplev = pd.read_excel('data/gdplev.xls', skiprows=219)
    gdplev.drop(['Unnamed: 3', 'Unnamed: 7'], axis=1,inplace=True)
    gdplev.columns = ['year', 'gdp in billions of current dollars', 'gdp in billions of chained 2009 dollars', 'year.1','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']
    gdp_quarter = gdplev[['year.1','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']]
    for i in range(2, len(gdp_quarter)):
        if (gdp_quarter.iloc[i-2][1] > gdp_quarter.iloc[i-1][1]) and (gdp_quarter.iloc[i-1][1] > gdp_quarter.iloc[i][1]):
            return gdp_quarter.iloc[i-2][0]
get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    gdplev = pd.read_excel('data/gdplev.xls', skiprows=219)
    gdplev.drop(['Unnamed: 3', 'Unnamed: 7'], axis=1,inplace=True)
    gdplev.columns = ['year', 'gdp in billions of current dollars', 'gdp in billions of chained 2009 dollars', 'quarter','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']
    gdp_quarter = gdplev[['quarter','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']]
    start = gdp_quarter.loc[gdp_quarter['quarter'] == get_recession_start()].index.values[0];
    for i in range(start + 2, len(gdp_quarter)):
        if (gdp_quarter.iloc[i-2][1] < gdp_quarter.iloc[i-1][1]) and (gdp_quarter.iloc[i-1][1] < gdp_quarter.iloc[i][1]):
            return gdp_quarter.iloc[i-2][0]
get_recession_end()

'2009q2'

In [6]:
def get_recession_bottom():
    gdplev = pd.read_excel('data/gdplev.xls', skiprows=219)
    gdplev.drop(['Unnamed: 3', 'Unnamed: 7'], axis=1,inplace=True)
    gdplev.columns = ['year', 'gdp in billions of current dollars', 'gdp in billions of chained 2009 dollars', 'quarter','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']
    gdp_quarter = gdplev[['quarter','gdp in billions of current dollars.1', 'gdp in billions of chained 2009 dollars.1']]
    start = gdp_quarter.loc[gdp_quarter['quarter'] == get_recession_start()].index.values[0];
    end = gdp_quarter.loc[gdp_quarter['quarter'] == get_recession_end()].index.values[0];
    
    min_value = gdp_quarter.iloc[start:end+1]['gdp in billions of current dollars.1'].idxmin()
    return gdp_quarter.iloc[min_value]['quarter']
    
get_recession_bottom()

'2009q2'

In [8]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

def get_columns():
    columns = ['RegionName', 'State']
    for year in range(2000, 2017):
        for quarter in range(1, 5):
            columns.append(str(year) + '-0' + str(quarter));
    return columns[:-1]


In [54]:
def convert_housing_data_to_quarters():
    housing = pd.read_csv('data/City_Zhvi_AllHomes.csv')
    housing = housing[get_columns()]
    housing.State = housing.State.apply(lambda x: states[x])
    housing = housing.sort_values(['State', 'RegionName']).reset_index()
    housing.drop(['index'], axis=1,inplace=True)
    housing.set_index(['State', 'RegionName'], drop=False, inplace=True)
    return housing
convert_housing_data_to_quarters().head()

RegionName    State   2000-01   2000-02   2000-03  \
State   RegionName                                                        
Alabama Adamsville    Adamsville  Alabama   68700.0   69100.0   69300.0   
        Alabaster      Alabaster  Alabama  121600.0  122300.0  122500.0   
        Albertville  Albertville  Alabama   74300.0   74100.0   73500.0   
        Arab                Arab  Alabama   84500.0   83900.0   82900.0   
        Ardmore          Ardmore  Alabama       NaN       NaN       NaN   

                      2000-04   2001-01   2001-02   2001-03   2001-04  \
State   RegionName                                                      
Alabama Adamsville    69200.0   73000.0   73400.0   74000.0   74100.0   
        Alabaster    122800.0  123200.0  123000.0  123500.0  124200.0   
        Albertville   72700.0   75700.0   75900.0   76100.0   76300.0   
        Arab          81900.0   83800.0   84200.0   84600.0   84800.0   
        Ardmore           NaN       NaN       NaN       NaN       NaN   

                      ...     2014-02  2014-03  2014-04  2015-01  2015-02  \
State   RegionName    ...                                                   
Alabama Adamsville    ...       78700    78000    77200    71000    71500   
        Alabaster     ...      146600   147000   147100   148800   148900   
        Albertville   ...       84100    84100    84000    88300    88400   
        Arab          ...      112300   112700   113200   111100   110000   
        Ardmore       ...      140000   140200   140500   142200   143300   

                     2015-03  2015-04  2016-01  2016-02  2016-03  
State   RegionName                                                
Alabama Adamsville     72500    73000    73600    74100    74800  
        Alabaster     149000   149300   152800   153400   154200  
        Albertville    88700    89100    91400    92000    92600  
        Arab          109400   109300   111000   109800   109300  
        Ardmore       144200   143700   142200   142500   143000  

[5 rows x 69 columns]

In [60]:
def run_ttest():
    housing_data = convert_housing_data_to_quarters()
    housing_region = convert_housing_data_to_quarters()['RegionName']
    university_region = get_list_of_university_towns()['RegionName']
    housing_data['is_university_town'] = housing_region.apply(lambda x: 1 if x in list(university_region) else 0)
    
    uni_town = housing_data.loc[housing_data.is_university_town == 1]
    non_uni = housing_data.loc[housing_data.is_university_town == 0]

    start = get_recession_start().replace('q', '-0')
    bottom = get_recession_bottom().replace('q', '-0')
    cols = [start, bottom, 'is_university_town']
    h_data = housing_data[cols]
    
    uni_towns = h_data[h_data.is_university_town == 1].dropna()
    non_uni_towns = h_data[h_data.is_university_town == 0].dropna()
    
    uni_towns['compare_value'] = uni_towns.apply(lambda row: (row[start] - row[bottom])/row[start],axis=1)
    non_uni_towns['compare_value'] = non_uni_towns.apply(lambda row: (row[start] - row[bottom])/row[start],axis=1)
    
    mean_uni = uni_towns['compare_value'].mean()
    mean_non_uni = non_uni_towns['compare_value'].mean()
    
    better = 'non-university town' if mean_uni < mean_non_uni else 'university town'
    
    ttest = ttest_ind(non_uni_towns['compare_value'], uni_towns['compare_value'])
    return (ttest.pvalue < 0.01, ttest.pvalue, better)

run_ttest()


(True, 9.942850139786513e-05, 'non-university town')